<a href="https://colab.research.google.com/github/thecybersahil/Detection-Of-Fake-News/blob/main/Fake_News_Detection_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the Dataset:
1. id: unique id for a news article
2. title: the title of a news article
3. author: author name of the news article
4. text: the text of the article
5. label: a label that marks whether the news article is real or fake
             1:Fake News
             2:Real News

Import dependencies

In [ ]:
import numpy as np

In [ ]:
import re

In [ ]:
import pandas as pd


In [ ]:
from nltk.corpus import stopwords

In [ ]:
from nltk.stem.porter import PorterStemmer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Printing stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-Processing


In [ ]:
from google.colab import files


uploaded = files.upload()


Saving training to training


In [ ]:
# Loading dataset to pandas dataFrame
news_dataset = pd.read_csv('/content/training')

In [ ]:
news_dataset.shape

(4009, 4)

In [ ]:
#print 1st 5 rows of dataset
news_dataset.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [ ]:
#counting no. of missing values in dataset
news_dataset.isnull().sum()

URLs         0
Headline     0
Body        21
Label        0
dtype: int64

In [ ]:
#replacing null values with empty string
news_dataset = news_dataset.fillna('')

In [ ]:
#merging 2 columns together
news_dataset['content'] = news_dataset['URLs']+' '+news_dataset['Headline']

In [ ]:
print(news_dataset['content'])

0       http://www.bbc.com/news/world-us-canada-414191...
1       https://www.reuters.com/article/us-filmfestiva...
2       https://www.nytimes.com/2017/10/09/us/politics...
3       https://www.reuters.com/article/us-mexico-oil-...
4       http://www.cnn.com/videos/cnnmoney/2017/10/08/...
                              ...                        
4004    http://beforeitsnews.com/sports/2017/09/trends...
4005    http://beforeitsnews.com/u-s-politics/2017/10/...
4006    https://www.activistpost.com/2017/09/ron-paul-...
4007    https://www.reuters.com/article/us-china-pharm...
4008    http://beforeitsnews.com/u-s-politics/2017/10/...
Name: content, Length: 4009, dtype: object


In [ ]:
#seperating data and label
X = news_dataset.drop(columns='Label', axis=1)
Y = news_dataset['Label']

In [ ]:
print(X)
print(Y)

                                                   URLs  ...                                            content
0     http://www.bbc.com/news/world-us-canada-414191...  ...  http://www.bbc.com/news/world-us-canada-414191...
1     https://www.reuters.com/article/us-filmfestiva...  ...  https://www.reuters.com/article/us-filmfestiva...
2     https://www.nytimes.com/2017/10/09/us/politics...  ...  https://www.nytimes.com/2017/10/09/us/politics...
3     https://www.reuters.com/article/us-mexico-oil-...  ...  https://www.reuters.com/article/us-mexico-oil-...
4     http://www.cnn.com/videos/cnnmoney/2017/10/08/...  ...  http://www.cnn.com/videos/cnnmoney/2017/10/08/...
...                                                 ...  ...                                                ...
4004  http://beforeitsnews.com/sports/2017/09/trends...  ...  http://beforeitsnews.com/sports/2017/09/trends...
4005  http://beforeitsnews.com/u-s-politics/2017/10/...  ...  http://beforeitsnews.com/u-s-politics/2017

Stemming: Reducing a word to its root word

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0       http www bbc com news world us canada four way...
1       http www reuter com articl us filmfestiv londo...
2       http www nytim com us polit corker blast trump...
3       http www reuter com articl us mexico oil carde...
4       http www cnn com video cnnmoney snl jason alde...
                              ...                        
4004    http beforeitsnew com sport trend watch html t...
4005    http beforeitsnew com u polit trump jr soon gi...
4006    http www activistpost com ron paul trump anarc...
4007    http www reuter com articl us china pharmaceut...
4008    http beforeitsnew com u polit vice presid mike...
Name: content, Length: 4009, dtype: object


In [ ]:
#seperating the data and label
X= news_dataset['content'].values
Y= news_dataset['Label'].values

In [ ]:
print(X)

['http www bbc com news world us canada four way bob corker skewer donald trump'
 'http www reuter com articl us filmfestiv london lastflagfli linklat war veteran comedi speak modern america say star iduskbn cd x linklat war veteran comedi speak modern america say star'
 'http www nytim com us polit corker blast trump republican nod agreement html rref collect fsectioncollect fpolit trump fight corker jeopard legisl agenda'
 ...
 'http www activistpost com ron paul trump anarch altright html ron paul trump anarch altright'
 'http www reuter com articl us china pharmaceut china accept oversea trial data bid speed drug approv iduskbn ce china accept oversea trial data bid speed drug approv'
 'http beforeitsnew com u polit vice presid mike penc leav nfl game anti american protest html vice presid mike penc leav nfl game anti american protest']


In [ ]:
print(Y)

[1 1 1 ... 0 1 0]


In [ ]:
Y.shape

(4009,)

In [ ]:
#converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X= vectorizer.transform(X)

In [ ]:
print(X)

  (0, 6142)	0.0913707965861414
  (0, 6110)	0.19516601163580974
  (0, 5993)	0.3433706840412274
  (0, 5835)	0.13471132314515255
  (0, 5697)	0.17541868111462783
  (0, 5058)	0.47251179415624456
  (0, 3763)	0.19551920592657707
  (0, 2674)	0.05492152293496671
  (0, 2151)	0.3433706840412274
  (0, 1584)	0.2764676222993295
  (0, 1181)	0.3316408691933927
  (0, 1079)	0.05608421956461582
  (0, 801)	0.2741946832006727
  (0, 610)	0.321980685921162
  (0, 455)	0.18948517165846368
  (1, 6142)	0.03920731462839173
  (1, 5970)	0.24365392494612786
  (1, 5891)	0.32965182657095965
  (1, 5835)	0.05780478476598985
  (1, 5228)	0.27632457040636355
  (1, 5166)	0.3401694252227138
  (1, 4834)	0.20296341899540832
  (1, 4639)	0.06652121675369806
  (1, 3581)	0.3464632642638293
  (1, 3261)	0.15374938988123907
  :	:
  (4007, 2674)	0.022860313203842745
  (4007, 1631)	0.2955076414994694
  (4007, 1333)	0.2797413184554806
  (4007, 1079)	0.023344269359725466
  (4007, 963)	0.3679175992235736
  (4007, 886)	0.10021713581785203


Splitting dataset to train and test data


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2, stratify=Y, random_state=2)

Trining the model: Logistic Regression Model

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

Accuracy Score

In [ ]:
# acccuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print ('Accuracy score of the training data: ',training_data_accuracy)

Accuracy score of the training data:  0.9987527284066106


In [ ]:
# acccuracy score on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print ('Accuracy score of the test data: ',test_data_accuracy)

Accuracy score of the test data:  0.9962593516209476


Making a predictive system

In [ ]:
X_new = X_test[24]

prediction = model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('This is Real News')
else:
  print('This is Fake News')

[1]
This is Fake News


In [ ]:
print(Y_test[24])

1
